## Import Packages

In [1]:
import sys
sys.path.append('..')

import warnings
warnings.filterwarnings("ignore")

from datetime import datetime
import pandas as pd
import sys
import os

from sklearn.model_selection import train_test_split

from detector.data_loader import LoadEnronData, LoadPhishingData, LoadSocEnggData
from detector.labeler import EnronLabeler, MismatchLabeler
from detector.preprocessor import Preprocessor
from utils.util_modeler import evaluate_and_log, get_f1_score, Augmentor

import wandb
import argparse

from sklearn.pipeline import Pipeline

from transformers import BertConfig, BertTokenizer, BertForSequenceClassification

import torch
import transformers
from torch.utils.data import TensorDataset
from transformers.data.processors.utils import InputExample
from transformers.data.processors.glue import glue_convert_examples_to_features



[nltk_data] Downloading package stopwords to
[nltk_data]     /common/home/ps1279/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [21]:
!export WANDB_CACHE_DIR='/freespace/local/ps1279/wandb/'

## Load Model

In [2]:
model_name = "bert-base-uncased"
config = BertConfig.from_pretrained(
    model_name,
    num_labels=3,
)
tokenizer = BertTokenizer.from_pretrained(
    model_name,
    do_lower_case=False,
)
model = BertForSequenceClassification.from_pretrained(
    model_name,
    config=config,
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
trainable_layers = [model.bert.encoder.layer[-1], model.bert.pooler, model.classifier]
total_params = 0
trainable_params = 0

for p in model.parameters():
        p.requires_grad = False
        total_params += p.numel()

for layer in trainable_layers:
    for p in layer.parameters():
        p.requires_grad = True
        trainable_params += p.numel()

print(f"Total parameters count: {total_params}")
print(f"Trainable parameters count: {trainable_params}")

Total parameters count: 109484547
Trainable parameters count: 7680771


In [4]:
MAX_SEQ_LENGHT = 128
LABEL_LIST = [0,1]

def _create_examples(df, set_type):
    """ Convert raw dataframe to a list of InputExample. Filter malformed examples
    """
    examples = []
    for index, row in df.iterrows():
        if row['Label'] not in LABEL_LIST:
            continue
        if not isinstance(row['Body'], str):
            continue
            
        guid = f"{index}-{set_type}"
        examples.append(
            InputExample(guid=guid, text_a=row['Body'], label=row['Label']))
    return examples

def _df_to_features(df, set_type):
    """ Pre-process text. This method will:
    1) tokenize inputs
    2) cut or pad each sequence to MAX_SEQ_LENGHT
    3) convert tokens into ids
    
    The output will contain:
    `input_ids` - padded token ids sequence
    `attention mask` - mask indicating padded tokens
    `token_type_ids` - mask indicating the split between premise and hypothesis
    `label` - label
    """
    examples = _create_examples(df, set_type)
    
    #backward compatibility with older transformers versions
    legacy_kwards = {}
    from packaging import version
    if version.parse(transformers.__version__) < version.parse("2.9.0"):
        legacy_kwards = {
            "pad_on_left": False,
            "pad_token": tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0],
            "pad_token_segment_id": 0,
        }
    
    return glue_convert_examples_to_features(
        examples=examples,
        tokenizer=tokenizer,
        label_list=LABEL_LIST,
        max_length=MAX_SEQ_LENGHT,
        output_mode="classification",
        **legacy_kwards,
    )

def _features_to_dataset(features):
    """ Convert features from `_df_to_features` into a single dataset
    """
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_attention_mask = torch.tensor(
        [f.attention_mask for f in features], dtype=torch.long
    )
    all_token_type_ids = torch.tensor(
        [f.token_type_ids for f in features], dtype=torch.long
    )
    all_labels = torch.tensor([f.label for f in features], dtype=torch.long)
    dataset = TensorDataset(
        all_input_ids, all_attention_mask, all_token_type_ids, all_labels
    )

    return dataset

## Init wandb for model tracking

In [5]:
wandbdict = {
    'key': os.getenv('WANDB_API_KEY'),
    'entity': os.getenv('WANDB_ENTITY'),
    'project': os.getenv('WANDB_PROJECT'),
}
wandb.login(key=wandbdict['key'])
run = wandb.init(project=wandbdict['project'], entity=wandbdict['entity'])

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: advaithrao (regressors). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /common/home/ps1279/.netrc


## Load Data

In [6]:
#The data file is too large to upload to github, so you will need to run from https://github.com/advaithsrao/Fraud-Detector/wiki/Load-Preprocessed-and-Labeled-Data#The data file is too large to upload to github, 
#so you will need to run data loading from https://github.com/advaithsrao/Fraud-Detector/wiki/Load-Preprocessed-and-Labeled-Data 
#and save it to <repo>/data/fraud_detector_data.csv
data = pd.read_csv('./data/fraud_detector_data.csv')

## Data Splits

In [7]:
train_data = data[data.Split == 'Train']
sanity_data = data[data.Split == 'Sanity']
gold_fraud_data = data[data.Split == 'Gold Fraud']

## Run Augmentation

In [12]:

augmentor = Augmentor()

train_body, train_labels = augmentor(
    train_data['Body'].tolist(),
    train_data['Label'].tolist(),
    aug_label=1,
    num_aug_per_label_1=9,
    shuffle=True
)

train_data = pd.DataFrame(
    {
        'Body': train_body,
        'Label': train_labels
    }
)

train_data.drop_duplicates(subset=['Body'], inplace=True)
train_data.reset_index(drop=True, inplace=True)


In [66]:
# train_data.to_csv('./data/augmented_train_data.csv', index=None)

## Train Model

In [9]:
# train_data = pd.read_csv('./data/augmented_train_data.csv')

In [10]:
train_features = _df_to_features(train_data, "train")
sanity_features = _df_to_features(sanity_data, "sanity")
gold_fraud_features = _df_to_features(gold_fraud_data, "gold_fraud")

all_train_dataset = _features_to_dataset(train_features)
sanity_dataset = _features_to_dataset(sanity_features)
gold_fraud_dataset = _features_to_dataset(gold_fraud_features)

In [11]:
# MAX_PHYSICAL_BATCH_SIZE defines the maximum batch size we can afford from a memory standpoint, and only affects computation speed
# BATCH_SIZE, on the other hand, will affect only convergence and privacy guarantee.
BATCH_SIZE = 32
MAX_PHYSICAL_BATCH_SIZE = 32

In [12]:
from torch.utils.data import DataLoader

val_size = int(0.1 * len(all_train_dataset))
train_size = len(all_train_dataset) - val_size
train_dataset, val_dataset = torch.utils.data.random_split(all_train_dataset, [train_size, val_size])

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
sanity_dataloader = DataLoader(sanity_dataset, batch_size=BATCH_SIZE)
gold_fraud_dataloader = DataLoader(gold_fraud_dataset, batch_size=BATCH_SIZE)

In [13]:
# Move the model to appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Set the model to train mode (HuggingFace models load in eval mode)
model = model.train()

# Define optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4, eps=1e-8)

In [14]:
EPOCHS = 5
LOGGING_INTERVAL = 5000 # once every how many steps we run evaluation cycle and report metrics
EPSILON = 20 # Privacy budget
DELTA = 1 / len(train_dataloader) # Parameter for privacy accounting. Probability of not achieving privacy guarantees

In [15]:
from opacus import PrivacyEngine

MAX_GRAD_NORM = 0.1

privacy_engine = PrivacyEngine()

model, optimizer, train_dataloader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimizer,
    data_loader=train_dataloader,
    target_delta=DELTA,
    target_epsilon=EPSILON, 
    epochs=EPOCHS,
    max_grad_norm=MAX_GRAD_NORM,
)

In [16]:
# Evaluation function
import numpy as np
from tqdm.notebook import tqdm

def accuracy(preds, labels):
    return (preds == labels).mean()

# define evaluation cycle
def evaluate(model):    
    model.eval()

    loss_arr = []
    accuracy_arr = []
    
    for batch in val_dataloader:
        batch = tuple(t.to(device) for t in batch)

        with torch.no_grad():
            inputs = {'input_ids':      batch[0],
                      'attention_mask': batch[1],
                      'token_type_ids': batch[2],
                      'labels':         batch[3]}

            outputs = model(**inputs)
            loss, logits = outputs[:2]
            
            preds = np.argmax(logits.detach().cpu().numpy(), axis=1)
            labels = inputs['labels'].detach().cpu().numpy()
            
            loss_arr.append(loss.item())
            accuracy_arr.append(accuracy(preds, labels))
    
    model.train()
    return np.mean(loss_arr), np.mean(accuracy_arr)

In [17]:
from opacus.utils.batch_memory_manager import BatchMemoryManager

for epoch in range(1, EPOCHS+1):
    losses = []

    with BatchMemoryManager(
        data_loader=train_dataloader, 
        max_physical_batch_size=MAX_PHYSICAL_BATCH_SIZE, 
        optimizer=optimizer
    ) as memory_safe_data_loader:
        for step, batch in enumerate(tqdm(memory_safe_data_loader)):
            optimizer.zero_grad()

            batch = tuple(t.to(device) for t in batch)
            inputs = {'input_ids':      batch[0],
                    'attention_mask': batch[1],
                    'token_type_ids': batch[2],
                    'labels':         batch[3]}

            outputs = model(**inputs) # output = loss, logits, hidden_states, attentions

            loss = outputs[0]
            loss.backward()
            losses.append(loss.item())

            optimizer.step()

            if step > 0 and step % LOGGING_INTERVAL == 0:
                train_loss = np.mean(losses)
                eps = privacy_engine.get_epsilon(DELTA)

                eval_loss, eval_accuracy = evaluate(model)

                print(
                  f"Epoch: {epoch} | "
                  f"Step: {step} | "
                  f"Train loss: {train_loss:.3f} | "
                  f"Eval loss: {eval_loss:.3f} | "
                  f"Eval accuracy: {eval_accuracy:.3f} | "
                  f"ɛ: {eps:.2f}"
                )

                wandb.log({
                    'Epoch': epoch,
                    'Step': step,
                    'Train Loss': train_loss,
                    'Eval Loss': eval_loss,
                    'Eval Accuracy': eval_accuracy,
                    'Epsilon': eps
                })

  0%|          | 0/6565 [00:00<?, ?it/s]

Epoch: 1 | Step: 5000 | Train loss: 0.747 | Eval loss: 0.641 | Eval accuracy: 0.908 | ɛ: 9.92


  0%|          | 0/6565 [00:00<?, ?it/s]

Epoch: 2 | Step: 5000 | Train loss: 0.560 | Eval loss: 0.608 | Eval accuracy: 0.916 | ɛ: 13.24


  0%|          | 0/6565 [00:00<?, ?it/s]

Epoch: 3 | Step: 5000 | Train loss: 0.560 | Eval loss: 0.641 | Eval accuracy: 0.917 | ɛ: 15.54


  0%|          | 0/6565 [00:00<?, ?it/s]

Epoch: 4 | Step: 5000 | Train loss: 0.550 | Eval loss: 0.564 | Eval accuracy: 0.930 | ɛ: 17.50


  0%|          | 0/6565 [00:00<?, ?it/s]

Epoch: 5 | Step: 5000 | Train loss: 0.524 | Eval loss: 0.570 | Eval accuracy: 0.926 | ɛ: 19.23


In [56]:
f1_scores = {}
os.makedirs('/common/home/ps1279/models/bert/logs', exist_ok=True)
save_path='/common/home/ps1279/models/bert/'

## Predict on all datasets and generate logs + mismatch_data

In [45]:
def predict(model, text_list, batch_size=32):
    model.eval()

    # Load tokenizer
    tokenizer = BertTokenizer.from_pretrained("bert-base-cased")  # Replace with the actual model name

    all_preds = []

    if len(text_list) < batch_size:
        batch_size = len(text_list)

    for i in range(0, len(text_list), batch_size):
        batch_text = text_list[i:i+batch_size]
        encoded_data = tokenizer(batch_text, padding=True, truncation=True, return_tensors='pt')
        encoded_data = {key: value.to(device) for key, value in encoded_data.items()}


        with torch.no_grad():
            outputs = model(**encoded_data)

        logits = outputs.logits  # Assuming logits is a tensor in the outputs
        preds = np.argmax(logits.detach().cpu().numpy(), axis=1)
        all_preds.extend(preds)

    return all_preds

In [51]:
predict(model,['This is certainly a fruad','hello sur','I am your temamate'])

[1, 0, 0]

In [53]:
# train_data.to_csv('/freespace/local/ps1279/model/bert_train_data.csv', index=None)

In [57]:
train_data['Prediction'] = predict(model, train_data['Body'].tolist())


evaluate_and_log(x=train_data['Body'].tolist(), y_true=train_data['Label'].tolist(), y_pred=train_data['Prediction'].tolist(), filename=os.path.join(save_path,'logs/train.log'), experiment=run)#, id = train_data['Mail-ID'].tolist())
f1_scores['train'] = get_f1_score(y_true=train_data['Label'].tolist(), y_pred=train_data['Prediction'].tolist())

wandb: WARNING Serializing object of type str that is 105009 bytes
wandb: WARNING Serializing object of type str that is 219224 bytes
wandb: WARNING Serializing object of type str that is 145414 bytes
wandb: WARNING Serializing object of type str that is 249676 bytes
wandb: WARNING Serializing object of type str that is 113670 bytes
wandb: WARNING Serializing object of type str that is 111211 bytes
wandb: WARNING Serializing object of type str that is 171024 bytes
wandb: WARNING Serializing object of type str that is 135282 bytes
wandb: WARNING Serializing object of type str that is 111007 bytes
wandb: WARNING Serializing object of type str that is 122873 bytes
wandb: WARNING Serializing object of type str that is 212448 bytes
wandb: WARNING Serializing object of type str that is 215807 bytes
wandb: WARNING Serializing object of type str that is 160129 bytes
wandb: WARNING Serializing object of type str that is 168313 bytes
wandb: WARNING Serializing object of type str that is 102369 b

In [58]:
sanity_data['Prediction'] = predict(model, sanity_data['Body'].tolist())

evaluate_and_log(x=sanity_data['Body'].tolist(), y_true=sanity_data['Label'].tolist(), y_pred=sanity_data['Prediction'].tolist(), filename=os.path.join(save_path,'logs/sanity.log'), experiment=run)#, id = sanity_data['Mail-ID'].tolist())
f1_scores['sanity'] = get_f1_score(y_true=sanity_data['Label'].tolist(), y_pred=sanity_data['Prediction'].tolist())



12/06/2023 22:00:28:WARNING:Truncating wandb.Table object to 200000 rows.
wandb: WARNING Serializing object of type str that is 187632 bytes
wandb: WARNING Serializing object of type str that is 155017 bytes
wandb: WARNING Serializing object of type str that is 103116 bytes
wandb: WARNING Serializing object of type str that is 147814 bytes
wandb: WARNING Serializing object of type str that is 148323 bytes
wandb: WARNING Serializing object of type str that is 150417 bytes
wandb: WARNING Serializing object of type str that is 115734 bytes
wandb: WARNING Serializing object of type str that is 221932 bytes
wandb: WARNING Serializing object of type str that is 263387 bytes
wandb: WARNING Serializing object of type str that is 102577 bytes
wandb: WARNING Serializing object of type str that is 153008 bytes
wandb: WARNING Serializing object of type str that is 1191251 bytes
wandb: WARNING Serializing object of type str that is 184613 bytes
wandb: WARNING Serializing object of type str that is 

In [59]:
gold_fraud_data['Prediction'] = predict(model, gold_fraud_data['Body'].tolist())


evaluate_and_log(x=gold_fraud_data['Body'].tolist(), y_true=gold_fraud_data['Label'].tolist(), y_pred=gold_fraud_data['Prediction'].tolist(), filename=os.path.join(save_path,'logs/gold_fraud.log'), experiment=run)#, id = gold_fraud_data['Mail-ID'].tolist())
f1_scores['gold_fraud'] = get_f1_score(y_true=gold_fraud_data['Label'].tolist(), y_pred=gold_fraud_data['Prediction'].tolist())



In [60]:
f1_scores

{'train': 0.29669621419108405,
 'sanity': 0.04849109331334988,
 'gold_fraud': 0.9959839357429718}

In [61]:
#save mismatch data into a csv file
mismatch_data = pd.concat(
    [
        train_data[train_data['Prediction'] != train_data['Label']],
        sanity_data[sanity_data['Prediction'] != sanity_data['Label']],
        gold_fraud_data[gold_fraud_data['Prediction'] != gold_fraud_data['Label']]
    ],
    axis=0,
    ignore_index=True
)

mismatch_data.to_csv(os.path.join(save_path,'logs/mismatch_data.csv'), index=False)

## Save Logs

In [62]:
hyper_params = dict(
    model_name=model_name,
    max_seq_length=MAX_SEQ_LENGHT,
    batch_size=BATCH_SIZE,
    max_physical_batch_size=MAX_PHYSICAL_BATCH_SIZE,
    epsilon=EPSILON,
    delta=DELTA,
    max_grad_norm=MAX_GRAD_NORM
)

all_params = {**hyper_params, **f1_scores}
run.config.update(all_params)

In [63]:
logs_path = os.path.join(save_path,'logs')
log_artifact = wandb.Artifact("fraud-detector-logs", type="logs")
log_artifact.add_dir(logs_path)
run.use_artifact(log_artifact)

wandb: Adding directory to artifact (/common/home/ps1279/models/bert/logs)... Done. 33.2s


<Artifact QXJ0aWZhY3Q6NjU5ODQ1ODI1>

## Save Model

In [20]:
from mlflow.sklearn import save_model
save_model(model, os.path.join(save_path,'model'))

In [21]:
model_path = os.path.join(save_path, 'model')
model_artifact = wandb.Artifact("fraud-detector-model", type="model")
model_artifact.add_dir(model_path)
run.use_artifact(model_artifact)

wandb: Adding directory to artifact (/common/home/ps1279/models/rf_diff_privacy/2023-12-05/rf_diff_privacy/model)... Done. 12.1s


<Artifact QXJ0aWZhY3Q6NjU5Mjg0NzIz>

In [22]:
# !rm -rf /common/home/ps1279/models/rf_diff_privacy/

In [64]:
run.finish()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Epoch,▁▃▅▆█
Epsilon,▁▃▅▇█
Eval Accuracy,▁▃▄█▇
Eval Loss,█▅█▁▂
Step,▁▁▁▁▁
Train Loss,█▂▂▂▁
Epoch,5
Epsilon,19.22647
Eval Accuracy,0.92641
Eval Loss,0.56994
Step,5000
